# Lecture 4 - Exploratory data analysis, training models

## The plan

- Business context of data
- Feature exploration
  - Categoricals
	- Support
  - Scaling
- (Outliers, missing data)
- Correlations
- Fitting a model in scikit-learn
- Evalutating a model

<img src="img/presentation.jpeg" style="width:40vw">

# The Business Context of Data

- Always need subject matter expertise, not obvious from data alone
  - Subject matter experts (SMEs) usually part of a project team
- In the lectures, we need to 'invent' the business context

### It is essential to get this right.
### If not, you will solve the wrong problem!


# Reading the data

# Buisness Cases

- Customized adertising
  - Marketing the right product to the right audience
    - Positives, and negatives. Aggressive marketing vs. wasteing people's time?
- Evaluating credit risk
- Quantifying career advise
  - Biases?

# Problems With a Data Set Like This

- Introducing biases
  - E.g. racial and gender biases
- Neglecting privacy
- GDPR?

## Try asking Chat GPT

Reflect on which problems you see with a dataset like this. Now ask ChatGPT which problems it sees. Are there any you would not have thought of?

    A dataset is collected containing demorgraphic information, such as age, gender, race, as well as education, occupation, hors worked per week, and income data. What potential business cases could be served with such a dataset? What problems do you see, ethical and practical?

## DISCLAIMER

In a real-world application, samples should be weighted by the fnlwgt column. We will ignore this here at our own peril.

## General shape of the data, correlations etc.

## Outliers

# An example business case - Advertising

- A company makes 5 USD profit per conversion, with a conversion rate of 0.1 for those in a 'high income' class.
- If we have a budget to target 10k people, how would our profit cahnge if we could could identify the 'Exec-managerial' group?

## Base Case

## Execl-managerial only

## Can we do better, by predicting the class with a multivariate model?

# Updated business case:

### Q: Why is this not performing better than our one-occupation case?

## Even better business case:

- What if you start to factor in cost of advertising?
- How does your business case depend on the conversion rate, for both groups?
- Advanced: Can you segment users who have a particularly high conversion rate?

## If Time: A cautionary exampled

## Optional: Pipelines
